# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 19 2022 1:36PM,244587,21,624598,"NBTY Global, Inc.",Released
1,Aug 19 2022 1:35PM,244586,19,8423501,"Emersa Waterbox, LLC",Released
2,Aug 19 2022 1:35PM,244586,19,8423502,"Emersa Waterbox, LLC",Released
3,Aug 19 2022 1:28PM,244585,21,6246112,"NBTY Global, Inc.",Released
4,Aug 19 2022 1:11PM,244582,22,624875,"NBTY Global, Inc.",Released
5,Aug 19 2022 1:10PM,244581,22,624830,"NBTY Global, Inc.",Released
6,Aug 19 2022 12:54PM,244579,10,SNAP31238569,"Snap Medical Industries, LLC",Released
7,Aug 19 2022 12:54PM,244579,10,SNAP31277751,"Snap Medical Industries, LLC",Released
8,Aug 19 2022 12:41PM,244570,10,0085942524,ISDIN Corporation,Executing
9,Aug 19 2022 12:41PM,244570,10,0085942530,ISDIN Corporation,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,244581,Released,1
19,244582,Released,1
20,244585,Released,1
21,244586,Released,2
22,244587,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244581,NaN,1.0
244582,NaN,1.0
244585,NaN,1.0
244586,NaN,2.0
244587,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244517,0.0,2.0
244530,1.0,6.0
244544,0.0,5.0
244546,1.0,0.0
244551,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244517,0,2
244530,1,6
244544,0,5
244546,1,0
244551,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244517,0,2
1,244530,1,6
2,244544,0,5
3,244546,1,0
4,244551,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244517,,2
1,244530,1,6
2,244544,,5
3,244546,1,
4,244551,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 19 2022 1:36PM,244587,21,"NBTY Global, Inc."
1,Aug 19 2022 1:35PM,244586,19,"Emersa Waterbox, LLC"
3,Aug 19 2022 1:28PM,244585,21,"NBTY Global, Inc."
4,Aug 19 2022 1:11PM,244582,22,"NBTY Global, Inc."
5,Aug 19 2022 1:10PM,244581,22,"NBTY Global, Inc."
6,Aug 19 2022 12:54PM,244579,10,"Snap Medical Industries, LLC"
8,Aug 19 2022 12:41PM,244570,10,ISDIN Corporation
20,Aug 19 2022 12:18PM,244567,10,"Methapharm, Inc."
24,Aug 19 2022 12:18PM,244568,10,Emerginnova
25,Aug 19 2022 12:16PM,244565,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 19 2022 1:36PM,244587,21,"NBTY Global, Inc.",,1
1,Aug 19 2022 1:35PM,244586,19,"Emersa Waterbox, LLC",,2
2,Aug 19 2022 1:28PM,244585,21,"NBTY Global, Inc.",,1
3,Aug 19 2022 1:11PM,244582,22,"NBTY Global, Inc.",,1
4,Aug 19 2022 1:10PM,244581,22,"NBTY Global, Inc.",,1
5,Aug 19 2022 12:54PM,244579,10,"Snap Medical Industries, LLC",,2
6,Aug 19 2022 12:41PM,244570,10,ISDIN Corporation,5,7
7,Aug 19 2022 12:18PM,244567,10,"Methapharm, Inc.",,4
8,Aug 19 2022 12:18PM,244568,10,Emerginnova,,1
9,Aug 19 2022 12:16PM,244565,10,"Methapharm, Inc.",,19


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 1:36PM,244587,21,"NBTY Global, Inc.",1,
1,Aug 19 2022 1:35PM,244586,19,"Emersa Waterbox, LLC",2,
2,Aug 19 2022 1:28PM,244585,21,"NBTY Global, Inc.",1,
3,Aug 19 2022 1:11PM,244582,22,"NBTY Global, Inc.",1,
4,Aug 19 2022 1:10PM,244581,22,"NBTY Global, Inc.",1,
5,Aug 19 2022 12:54PM,244579,10,"Snap Medical Industries, LLC",2,
6,Aug 19 2022 12:41PM,244570,10,ISDIN Corporation,7,5
7,Aug 19 2022 12:18PM,244567,10,"Methapharm, Inc.",4,
8,Aug 19 2022 12:18PM,244568,10,Emerginnova,1,
9,Aug 19 2022 12:16PM,244565,10,"Methapharm, Inc.",19,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 1:36PM,244587,21,"NBTY Global, Inc.",1,
1,Aug 19 2022 1:35PM,244586,19,"Emersa Waterbox, LLC",2,
2,Aug 19 2022 1:28PM,244585,21,"NBTY Global, Inc.",1,
3,Aug 19 2022 1:11PM,244582,22,"NBTY Global, Inc.",1,
4,Aug 19 2022 1:10PM,244581,22,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 1:36PM,244587,21,"NBTY Global, Inc.",1.0,NaN
1,Aug 19 2022 1:35PM,244586,19,"Emersa Waterbox, LLC",2.0,NaN
2,Aug 19 2022 1:28PM,244585,21,"NBTY Global, Inc.",1.0,NaN
3,Aug 19 2022 1:11PM,244582,22,"NBTY Global, Inc.",1.0,NaN
4,Aug 19 2022 1:10PM,244581,22,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 19 2022 1:36PM,244587,21,"NBTY Global, Inc.",1.0,0.0
1,Aug 19 2022 1:35PM,244586,19,"Emersa Waterbox, LLC",2.0,0.0
2,Aug 19 2022 1:28PM,244585,21,"NBTY Global, Inc.",1.0,0.0
3,Aug 19 2022 1:11PM,244582,22,"NBTY Global, Inc.",1.0,0.0
4,Aug 19 2022 1:10PM,244581,22,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1956491,40.0,8.0
16,1222737,10.0,0.0
17,244560,1.0,0.0
19,244586,2.0,0.0
21,978276,3.0,1.0
22,489163,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1956491,40.0,8.0
1,16,1222737,10.0,0.0
2,17,244560,1.0,0.0
3,19,244586,2.0,0.0
4,21,978276,3.0,1.0
5,22,489163,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,40.0,8.0
1,16,10.0,0.0
2,17,1.0,0.0
3,19,2.0,0.0
4,21,3.0,1.0
5,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,40.0
1,16,Released,10.0
2,17,Released,1.0
3,19,Released,2.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,17,19,21,22
Status,,,,,,
Executing,8.0,0.0,0.0,0.0,1.0,0.0
Released,40.0,10.0,1.0,2.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,17,19,21,22
0,Executing,8.0,0.0,0.0,0.0,1.0,0.0
1,Released,40.0,10.0,1.0,2.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,17,19,21,22
0,Executing,8.0,0.0,0.0,0.0,1.0,0.0
1,Released,40.0,10.0,1.0,2.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()